In [ ]:
!pip install datasets
!pip install transformers
!pip install sentence_transformers
!pip install peft
!pip install bitsandbytes


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 19.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 20.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.8 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.3/245.3 kB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.5/322.5 kB 22.4 MB/s eta 0:00:00

In [ ]:
!pip install trl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.4/318.4 kB 18.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.9/105.9 kB 11.1 MB/s eta 0:00:00


In [ ]:
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, TrainingArguments, Trainer, GenerationConfig, AutoModelForCausalLM, BitsAndBytesConfig, AutoConfig
import time
from sentence_transformers import SentenceTransformer
from peft import LoraConfig, get_peft_model, TaskType
from peft import PeftModel, PeftConfig
import torch
from trl import SFTTrainer, SFTConfig

In [ ]:
ds = load_dataset("PandurangMopgar/fitness_data")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/345 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/107k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/245 [00:00<?, ? examples/s]

In [ ]:
ds = ds.filter(lambda example: example['instruction'] != 'instruction')


Filter:   0%|          | 0/245 [00:00<?, ? examples/s]

In [ ]:
ds

DatasetDict({
    train: Dataset({
        features: ['instruction', 'output', 'text'],
        num_rows: 170
    })
})

In [ ]:
def tokenize_function(example):
    start_prompt = 'Question:\n'
    end_prompt = '\n\nAnswer:\n '
    example['text'] = [f"{start_prompt}{inst}{end_prompt}{ans}" for inst, ans in zip(example["instruction"], example["output"])]
    return example

tokenized_ds = ds.map(tokenize_function, batched=True)


Map:   0%|          | 0/170 [00:00<?, ? examples/s]

In [ ]:
print(tokenized_ds)

DatasetDict({
    train: Dataset({
        features: ['instruction', 'output', 'text'],
        num_rows: 170
    })
})


In [ ]:
model_id = 'genaitraining/llama-2-7b-domain-tuned'
compute_dtype = getattr(torch, 'float16')

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=compute_dtype
)
tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)


tokenizer_config.json:   0%|          | 0.00/1.71k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/437 [00:00<?, ?B/s]

In [ ]:
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

In [ ]:
model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map='auto')
config = AutoConfig.from_pretrained(model_id)

config.json:   0%|          | 0.00/661 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/3.59G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/183 [00:00<?, ?B/s]

In [ ]:
peft_config = LoraConfig(
    r=64, # Rank
    lora_alpha=16,
    lora_dropout=0.1,
    bias="none",
    task_type="CAUSAL_LM"
)


In [ ]:
output_dir = "my_fine_tuned_llama2"
peft_training_args = TrainingArguments(
    output_dir=output_dir,
    hub_model_id="Sahaj10/my_fine_tuned_llama2",
    learning_rate=2e-4,
    per_device_train_batch_size=2,
    num_train_epochs=10,
    save_strategy = "epoch",
    fp16=True,
    seed = 42
)


trainer = SFTTrainer (
    model=model,
    train_dataset=tokenized_ds['train'],
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=512,
    tokenizer=tokenizer,
    args=peft_training_args,

)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': dataset_text_field, max_seq_length. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:283: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:321: UserWarning: You passed a `dataset_text_field` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(


Map:   0%|          | 0/170 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


In [ ]:
trainer.train()

Step,Training Loss
500,0.747200


TrainOutput(global_step=850, training_loss=0.5072140547808479, metrics={'train_runtime': 1176.3063, 'train_samples_per_second': 1.445, 'train_steps_per_second': 0.723, 'total_flos': 1.1627362447097856e+16, 'train_loss': 0.5072140547808479, 'epoch': 10.0})

In [ ]:
trainer.save_model('/content/drive/My Drive/trainer_saved')

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
trainer.push_to_hub()



Upload 3 LFS files:   0%|          | 0/3 [00:00<?, ?it/s]

adapter_model.safetensors:   0%|          | 0.00/134M [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/5.50k [00:00<?, ?B/s]

events.out.tfevents.1727394840.1af106f6a280.486.0:   0%|          | 0.00/6.36k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Sahaj10/my_fine_tuned_llama2/commit/58ca7d55b1b7e4cb5727fe4ddf17322e47c909c3', commit_message='End of training', commit_description='', oid='58ca7d55b1b7e4cb5727fe4ddf17322e47c909c3', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
config.push_to_hub("Sahaj10/my_fine_tuned_llama2")

README.md:   0%|          | 0.00/1.19k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Sahaj10/my_fine_tuned_llama2/commit/5bca3415a332666022e3daeab8ff35b459412207', commit_message='Upload config', commit_description='', oid='5bca3415a332666022e3daeab8ff35b459412207', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
del model
del trainer
import gc
gc.collect()
torch.cuda.empty_cache()

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

new_model = AutoModelForCausalLM.from_pretrained(
    'Sahaj10/my_fine_tuned_llama2', low_cpu_mem_usage=True,
    return_dict = True,
    torch_dtype = torch.float16).to(device)



Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
def prepare_input_for_inference(question):
    start_prompt = 'Question:\n'
    end_prompt = '\n\nAnswer:\n '
    return f"{start_prompt}{question}{end_prompt}"

question = 'How can I manage stress and maintain a healthy work-life balance?'
prepaperd_input = prepare_input_for_inference(question)
print(prepaperd_input)
input_ids = tokenizer(prepaperd_input, return_tensors="pt", truncation=True).input_ids.to(device)


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Question:
How can I manage stress and maintain a healthy work-life balance?

Answer:
 


In [ ]:
outputs = new_model.generate(input_ids=input_ids, max_new_tokens=200, temperature=0.6)


In [ ]:
result = tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True)[0]

In [ ]:
print(result)

Question:
How can I manage stress and maintain a healthy work-life balance?

Answer:
 1. Set clear boundaries: Make sure you have time for your personal life and family. Set boundaries around your work hours and days so you can have time to yourself and with your family.

2. Take breaks: Take short breaks throughout the day. Go for a walk, stretch, or do some deep breathing exercises.

3. Prioritize tasks: Make a list of the most important tasks and tackle the most important ones first. This will help you prepare for the next day and reduce stress.

4. Outsource tasks: If you have too much on your plate, consider outsourcing some tasks to free up time and energy.

5. Make time for self-care: Make time for self-care activities like exercise, meditation, and yoga. These activities can help reduce stress and maintain healthy work-life balance.

6. Disconnect: Take time off to disconnect from work. This can
